<a href="https://colab.research.google.com/github/CPukszta/gitbrachmerge/blob/main/homework/hw7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 7.2: MLE of microtubule catastrophe data (65 pts)

[Dataset download](https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_time_to_catastrophe_dic_tidy.csv)

<hr>

Refresh yourself about the microtubule catastrophe data we explored in previous homeworks. We will again work with this data set here.

**a)** In their [paper](http://dx.doi.org/10.1016/j.cell.2011.10.037), Gardner, Zanic, and coworkers modeled microtubule catastrophe times as Gamma distributed. Perform a maximum likelihood estimate for the parameters of the Gamma distribution. Because you showed in a previous homework that there is little difference between labeled and unlabeled tubulin, you only need to work this out for the labeled tubulin now and in part (b). Be sure to include confidence intervals or a confidence region for your MLE and discuss the method you used to get the confidence intervals or confidence region.


In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot bebi103 watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------

In [2]:
import numpy as np
import pandas as pd

import bokeh.io 
import bokeh.plotting
!pip install iqplot
import iqplot

import scipy.stats
import warnings
import tqdm

bokeh.io.output_notebook()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We will first read in the tubulin data!

We will only be using the labeled tubulin because as shown previously, there is little difference between the two. 

In [3]:
tubulin = pd.read_csv(os.path.join(data_path, "gardner_time_to_catastrophe_dic_tidy.csv"), header=0)

tub_lab = tubulin.loc[tubulin["labeled"]== True]
ttc = np.array(tub_lab["time to catastrophe (s)"].tolist())

We will now define our MLE for the gamma distribution

In [4]:
def gamma_log_like(params, t):
  a , b = params
  if (a <= 0) or (b <= 0):
    return -np.inf
  
  return np.sum(scipy.stats.gamma.logpdf(t,a=a,scale=1/b))

We will need to have some inital guess at the values of alpha and beta. There isnt a great way to calcualte these so we will have to guess!

In [5]:
alpha_guess = 2
beta_guess = 1

now lets minimze it to find alpha and beta

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    res = scipy.optimize.minimize(
        fun=lambda params, t: -gamma_log_like(params, ttc),
        x0=np.array([alpha_guess,beta_guess]),
        args=(ttc,),
        method='Powell',
        tol=1e-6,
      )

if res.success:
    alpha, beta = res.x
else:
    raise RuntimeError('Convergence failed with message', res.message)

print("alpha: ", alpha)
print("beta: ", beta)

alpha:  2.407546795530042
beta:  0.0054628788826684045


Now we can compute the Confidence interval on these perameters. We will start by defining a function to draw the parametric bootstraps and compute the MLE from them. 

In [7]:
# taken from lesson 21
def draw_parametric_bs_reps_mle(
    mle_fun, gen_fun, data, args=(), size=1, progress_bar=False
):
    """Draw parametric bootstrap replicates of maximum likelihood estimator.

    Parameters
    ----------
    mle_fun : function
        Function with call signature mle_fun(data, *args) that computes
        a MLE for the parameters
    gen_fun : function
        Function to randomly draw a new data set out of the model
        distribution parametrized by the MLE. Must have call
        signature `gen_fun(*params, size)`.
    data : one-dimemsional Numpy array
        Array of measurements
    args : tuple, default ()
        Arguments to be passed to `mle_fun()`.
    size : int, default 1
        Number of bootstrap replicates to draw.
    progress_bar : bool, default False
        Whether or not to display progress bar.

    Returns
    -------
    output : numpy array
        Bootstrap replicates of MLEs.
    """
    params = mle_fun(data, *args)

    if progress_bar:
        iterator = tqdm.tqdm(range(size))
    else:
        iterator = range(size)

    return np.array(
        [mle_fun(gen_fun(*params, size=len(data), *args)) for _ in iterator]
    )

We can now define a function to calculate the mle for each bootstrap replicate drawn above!




In [8]:
def mle_iid_gamma(t):
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    res = scipy.optimize.minimize(
        fun=lambda params, t: -gamma_log_like(params, t),
        x0=np.array([alpha,beta]),
        args=(t,),
        method='Powell',
        tol=1e-6,
      )

  if res.success:
      return res.x
  else:
      raise RuntimeError('Convergence failed with message', res.message)

We can now define our random number generator and return the generative function for the alpha and beta.

In [9]:
rg = np.random.default_rng()

def gen_gamma(alpha, beta, size):
  return rg.gamma(alpha, 1/beta, size=size)

Now we can actually run our code! (note this takes ~5 minutes)

In [10]:
bs_reps_parametric_labeled = draw_parametric_bs_reps_mle(
    mle_iid_gamma,
    gen_gamma,
    ttc,
    args=(),
    size=10000,
    progress_bar=True,
)

100%|██████████| 10000/10000 [05:04<00:00, 32.85it/s]


Finally, we can compute and print our confidence intervals.

In [11]:
lab_alpha = bs_reps_parametric_labeled[:,0]
lab_beta = bs_reps_parametric_labeled[:,1]

print("The 95% confidence interval for alpha is", np.percentile(lab_alpha, [2.5, 97.5]))
print("The 95% confidence interval for beta is", np.percentile(lab_beta, [2.5, 97.5]))

The 95% confidence interval for alpha is [2.04567712 2.93176438]
The 95% confidence interval for beta is [0.00455616 0.00677065]


We can see that our MLE estimates fit nicely within those confidence intervals :)

**b)** Obtain a maximum likelihood estimate for the parameters $\beta_1$ and $\beta_2$ from the model you derived in homework 5.2. As a reminder, you derived that the PDF for microtubule catastrophe times is

\begin{align}
f(t;\beta_1, \beta_2) = \frac{\beta_1\beta_2}{\beta_2 - \beta_1}\left(\mathrm{e}^{-\beta_1 t} - \mathrm{e}^{-\beta_2 t}\right).
\end{align}

**Be careful**; this is a *very* tricky calculation. It is possible to make significant analytical progress, and this can help you find the MLE. Be sure to think about what happens when $\beta_1 \approx \beta_2$. You may also need to think about how you will handle the [log of sums of exponentials](https://en.wikipedia.org/wiki/LogSumExp).


We will first start by defining our log sum exponential trick function.

In [12]:
def logsumexp_2(t, b1, delta_b):
  return np.sum(np.log(np.exp(-b1 * t)) + np.log(1 - np.exp(-delta_b*t)))

Now we can define our log likelihood function using the logsumexp trick defined above to ensure that we do not have issues with computing the function.

In [13]:
def log_like_func(params,t):
  b1, delta_b = params

  if (b1 <= 0) or (delta_b < 0):
    return -np.inf

  if (delta_b == 0):
    return np.sum(scipy.stats.gamma.logpdf(t,a=2,scale=1/b1))

  n = np.size(t)
  return n*np.log(b1) + n*np.log(delta_b + b1) - n*np.log(delta_b) + logsumexp_2(t,b1,delta_b)
  

Now that we have a function for the log likelyhood function, we can compute the MLEs. We will start by defining some inital guesses.

In [14]:
b1_guess = 0.001
b2_guess = 0.005
delta_b_guess = 0.00005

Now lets compute some MLEs!

In [15]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    res = scipy.optimize.minimize(
        fun=lambda params, t: -log_like_func(params, ttc),
        x0=np.array([b1_guess,delta_b_guess]),
        args=(ttc,),
        method='Powell',
        tol=1e-6,
      )

if res.success:
    beta1, delta_b = res.x
else:
    raise RuntimeError('Convergence failed with message', res.message)

print("beta_1: ", beta1)
print("delta beta: ", delta_b)
print("beta_2: ", beta1+delta_b)

beta_1:  0.004514882638033071
delta beta:  4.67205479025e-05
beta_2:  0.004561603185935572


Since $\Delta \beta$ is so small, let's try to approximate this as a gamma distribution with $\alpha$=2 and $\beta$=0.0045

In [16]:
p = iqplot.ecdf(data = tub_lab, q = "time to catastrophe (s)", 
                width = 800, height = 400,
                palette  = "Orange",
                conf_int = True, n_bs_reps = 10000, title = "ECDF of Actual Data with Overlaid Confidence Interval"
)
gamma_pts = rg.gamma(2, (1/beta1), size=1000)
gamma_plot = iqplot.ecdf(data=gamma_pts, q = "time to catastrophe (s)",
                         width = 800, height = 400,
                         palette = "Purple", title = "Gamma Approximatation ECDF ")
bokeh.io.show(p)
bokeh.io.show(gamma_plot)

this matches the actual data quite well! :)

**c)** Attempt to compute bootstrap confidence intervals for the parameters. This calculation has difficult subtleties that make it even trickier than computing the MLE. I suspect that most students will not be able to perform this calculation. We are asking for your best try here. You will be graded based on your effort and ability to identify where the difficulties in the confidence interval calculation arise.

Lets start by defining our generative distribution. We will randomly draw values from the generative distribution which we have solved for in 5.2.

In order to do this, we will need to use inverse transform sampling in which we generate n samples from a uniform(0,1), invert the CDF of the distribution, and finally plug the samples into the inverted CDF to get samples from the distribution. 

One big problem with this is that the PDF and CDF are both non-invertable! Therefore, we will use the taylor expansion of the two exponential terms and then invert it. Doing this with more than one term is possible, but gives a massive inverted function.

Our new PDF is: 

$\frac{\beta _1 \beta _2 \left(\beta _2
   x-\beta _1 x\right)}{\beta _2-\beta _1}$

When the PDF is integrated it gives the following CDF:

$F(t,\beta_1,\beta_2) = \frac{1}{2}{\beta _1 \beta _2
   t^2} $

Finally we can invert this function to get the inverse CDF:

   $F^{-1}(p;\beta_1,\beta_2) = \sqrt{\frac{2 p}{\beta _1 \beta _2}} $

   Where $F^{-1}(p)$ gives the time corresponding to a probaility between 0 and 1 given a certain value of $\beta_1$ and $\beta_2$.



In [17]:
def gen_betas(b1, delta_b, size=1000):
  """takes in a beta 1 and a delta beta and a size and generates 
   size samples from the generative distribution. 
  """

  b2 = delta_b + b1
  p = rg.random((1,size))
  return (np.sqrt(2*p))/(np.sqrt(b1*b2))


Lets take a look at this generative distribution using the MLE perameters that we have already computed and compare it to the actual CDF

In [18]:
def true_CDF(t,b1, delta_b):
    b2 = delta_b + b1
    num = (1-np.exp(-t*b2))*b1 + (-1+np.exp(-t*b1))*b2
    denom = delta_b
    return -num/denom

Now lets look at the distributions

In [19]:
betas_gen = gen_betas(beta1,delta_b,100000)
time = np.linspace(0,1200)
CDF = true_CDF(time,beta1,delta_b)

betas_plot = iqplot.ecdf(data=betas_gen, q = "time to catastrophe (s)",
                         width = 800, height = 400,
                         palette = "Purple", title = "betas generative Distribution")
bokeh.io.show(betas_plot)

c = bokeh.plotting.figure(
    width = 800, height = 400,
    title = "Actual CDF"
)

c.line(time,CDF)

bokeh.io.show(c)

This genetative distribution works for low times to catastrophe but dosent ever see the upper times. Its not a great distribtuion, but doing more terms of the taylor aproximation is not practical.

We will now define our mle function. 

In [20]:
def mle_iid_betas(t):
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    res = scipy.optimize.minimize(
        fun=lambda params, t: -log_like_func(params, t),
        x0=np.array([b1_guess, delta_b_guess]),
        args=(t,),
        method='Powell',
        tol=1e-6,
      )

  if res.success:
      return res.x
  else:
      raise RuntimeError('Convergence failed with message', res.message)

Now lets run the confidence interval! (note takes ~2 minutes to run)

In [21]:
bs_reps_parametric_labeled_betas = draw_parametric_bs_reps_mle(
    mle_iid_betas,
    gen_betas,
    ttc,
    args=(),
    size=10000,
    progress_bar=True,
)

100%|██████████| 10000/10000 [01:31<00:00, 109.63it/s]


Finally, lets compute and display the confidence intervals on $\beta_1$ and $\Delta \beta$

In [30]:
lab_beta_1 = bs_reps_parametric_labeled_betas[:,0]
lab_delta_beta = bs_reps_parametric_labeled_betas[:,1]


print("The 95% confidence interval for beta 1 is", np.percentile(lab_beta, [2.5, 97.5]))
print("The 95% confidence interval for delta beta is", np.percentile(lab_delta_beta, [2.5, 97.5]))

The 95% confidence interval for beta 1 is [0.00455616 0.00677065]
The 95% confidence interval for delta beta is [3.09225233e-05 3.28359741e-05]


Our confidence intervals are imperfect as they are only a Taylor series to first approximation. However these confidence intervals include our value of $\beta_1$ and are only off by $1e-05$ for our delta beta value which isn't bad. 

There may be more confidence intervals to find that fit the unlabeled tubulin data depending on the inital guesses used because there is the potential for many local minimia. Here, we have found the confidence interval that fits our perameter estimate. 

Attribution statement: Alex and Chris partner coded all of part A and started part B together. Alex went to OH Wed and finished part B. Chris wrote part C and added comments. Alex cleaned up the file and added comments.

In [23]:
%load_ext watermark
%watermark -v -p pandas,jupyterlab

Python implementation: CPython
Python version       : 3.7.15
IPython version      : 7.9.0

pandas    : 1.3.5
jupyterlab: not installed

